In [1027]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

import datetime
import pandas_ta as ta

import plotly.graph_objects as go


In [1028]:
# plt.rcParams['figure.figsize'] = [10, 7]

# plt.rc('font', size=14)

In [1029]:
# np.random.seed(0)

In [1030]:
# y = np.arange(0,100,1) + np.random.normal(0,10,100)

In [1031]:
# sma = pd.Series(y).rolling(20).mean()

In [1032]:
# plt.plot(y,label="Time series")
# plt.plot(sma,label="20-period SMA")

# plt.legend()
# plt.show()

In [1033]:
# n_forward = 40
# name = 'GLD'
# start_date = "2010-01-01"
# end_date = "2020-06-15"


In [1034]:
# # ticker = yfinance.Ticker("FB")
# # data = ticker.history(interval="1d",start='2010-01-01',end=end_date)

# # data=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\4h\RADUSDT_4h.csv", index_col="Time")
# n_forward = 40

# def best_sma(data):
#     data['Forward Close'] = data['Close'].shift(-n_forward)

#     data['Forward Return'] = (data['Forward Close'] - data['Close'])/data['Close']

#     result = []
#     train_size = 0.6

#     for sma_length in range(20,500):
#         data['ema'] = ta.ema(ta.ohlc4(data["Open"], data["High"], data["Low"], data["Close"]), length=sma_length)
    
#         # data['SMA'] = data['Close'].rolling(sma_length).mean()
#         # data['input'] = [int(x) for x in data['Close'] > data['SMA']]
#         data['input'] = [int(x) for x in data['Close'] > data['ema']]
        
#         df = data.dropna()

#         training = df.head(int(train_size * df.shape[0]))
#         test = df.tail(int((1 - train_size) * df.shape[0]))
        
#         tr_returns = training[training['input'] == 1]['Forward Return']
#         test_returns = test[test['input'] == 1]['Forward Return']

#         mean_forward_return_training = tr_returns.mean()
#         mean_forward_return_test = test_returns.mean()

#         pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
        
#         result.append({
#             'sma_length':sma_length,
#             'training_forward_return': mean_forward_return_training,
#             'test_forward_return': mean_forward_return_test,
#             'p-value':pvalue
#         })

#     result.sort(key = lambda x : -x['training_forward_return'])

#     best_sma = result[0]['sma_length']
#     # data['SMA'] = data['Close'].rolling(best_sma).mean()
#     data['ema'] = ta.ema(ta.ohlc4(data["Open"], data["High"], data["Low"], data["Close"]), length=best_sma)



#     candlestick = go.Candlestick(x=data['Time'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])

#     fig = go.Figure(data=candlestick)

#     fig.add_trace(go.Scatter(x=data['Time'], y=data['ema'], name=f"{best_sma} periods SMA"))
#     fig.layout.xaxis.rangeslider.visible = False

#     fig.show()




In [1035]:
# pair = "mdx".upper()
# data=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\4h\{pair}USDT_4h.csv", parse_dates=True)
# # data=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\1d\ENSUSDT_1d.csv", parse_dates=True)
# best_sma(data)

In [1036]:
n_forward = 40

def best_sma(data):
    data['Forward Low'] = data['Low'].shift(-n_forward)

    data['Forward Return'] = (data['Forward Low'] - data['Low'])/data['Low']

    result = []
    train_size = 0.6

    for sma_length in range(20,500):
    
        data['SMALow'] = data['Low'].rolling(sma_length).mean()
        data['input'] = [int(x) for x in data['Low'] > data['SMALow']]
        
        df = data.dropna()

        training = df.head(int(train_size * df.shape[0]))
        test = df.tail(int((1 - train_size) * df.shape[0]))
        
        tr_returns = training[training['input'] == 1]['Forward Return']
        test_returns = test[test['input'] == 1]['Forward Return']

        mean_forward_return_training = tr_returns.mean()
        mean_forward_return_test = test_returns.mean()

        pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
        
        result.append({
            'sma_length':sma_length,
            'training_forward_return': mean_forward_return_training,
            'test_forward_return': mean_forward_return_test,
            'p-value':pvalue
        })

    result.sort(key = lambda x : -x['training_forward_return'])

    best_smaLow = result[0]['sma_length']
    data['SMALow'] = data['Low'].rolling(best_smaLow).mean()



    data['Forward High'] = data['High'].shift(-n_forward)

    data['Forward Return'] = (data['Forward High'] - data['High'])/data['High']

    result = []
    train_size = 0.6

    for sma_length in range(20,500):
    
        data['SMAHigh'] = data['High'].rolling(sma_length).mean()
        data['input'] = [int(x) for x in data['High'] > data['SMAHigh']]
        
        df = data.dropna()

        training = df.head(int(train_size * df.shape[0]))
        test = df.tail(int((1 - train_size) * df.shape[0]))
        
        tr_returns = training[training['input'] == 1]['Forward Return']
        test_returns = test[test['input'] == 1]['Forward Return']

        mean_forward_return_training = tr_returns.mean()
        mean_forward_return_test = test_returns.mean()

        pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
        
        result.append({
            'sma_length':sma_length,
            'training_forward_return': mean_forward_return_training,
            'test_forward_return': mean_forward_return_test,
            'p-value':pvalue
        })

    result.sort(key = lambda x : -x['training_forward_return'])

    best_smaHigh = result[0]['sma_length']
    data['SMAHigh'] = data['High'].rolling(best_smaHigh).mean()


    # 

    data['Forward Close'] = data['Close'].shift(-n_forward)

    data['Forward Return'] = (data['Forward Close'] - data['Close'])/data['Close']

    result = []
    train_size = 0.6

    for sma_length in range(20,500):
    
        data['SMA'] = data['Close'].rolling(sma_length).mean()
        data['input'] = [int(x) for x in data['Close'] > data['SMA']]
        
        df = data.dropna()

        training = df.head(int(train_size * df.shape[0]))
        test = df.tail(int((1 - train_size) * df.shape[0]))
        
        tr_returns = training[training['input'] == 1]['Forward Return']
        test_returns = test[test['input'] == 1]['Forward Return']

        mean_forward_return_training = tr_returns.mean()
        mean_forward_return_test = test_returns.mean()

        pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
        
        result.append({
            'sma_length':sma_length,
            'training_forward_return': mean_forward_return_training,
            'test_forward_return': mean_forward_return_test,
            'p-value':pvalue
        })

    result.sort(key = lambda x : -x['training_forward_return'])

    best_sma = result[0]['sma_length']
    data['SMA'] = data['Close'].rolling(best_sma).mean()

    #

    data['SMA_C_Low'] = data['Close'].rolling(best_smaLow).mean()
    data['SMA_C_High'] = data['Close'].rolling(best_smaHigh).mean()




    candlestick = go.Candlestick(x=data['Time'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])

    fig = go.Figure(data=candlestick)

    # fig.add_trace(go.Scatter(x=data['Time'], y=data['SMALow'], name=f"{best_smaLow} Low SMA"))
    # fig.add_trace(go.Scatter(x=data['Time'], y=data['SMAHigh'], name=f"{best_smaHigh} High SMA"))
    fig.add_trace(go.Scatter(x=data['Time'], y=data['SMA'], name=f"{best_sma} Close SMA"))
    fig.add_trace(go.Scatter(x=data['Time'], y=data['SMA_C_Low'], name=f"{best_smaLow} clow SMA"))
    fig.add_trace(go.Scatter(x=data['Time'], y=data['SMA_C_High'], name=f"{best_smaHigh} chigh SMA"))
    fig.layout.xaxis.rangeslider.visible = False

    fig.show()

In [1037]:
pair = "mdx".upper()
data=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\4h\{pair}USDT_4h.csv", parse_dates=True)
# data=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\1d\ENSUSDT_1d.csv", parse_dates=True)
best_sma(data)

In [1038]:
# plt.plot(data['Close'],label='Facebook')


# plt.plot(data['Close'].rolling(20).mean(),label = "20-periods SMA")
# plt.plot(data['Close'].rolling(50).mean(),label = "50-periods SMA")
# plt.plot(data['Close'].rolling(200).mean(),label = "200-periods SMA")

# plt.legend()
# # plt.xlim((datetime.date(2021,1,1),datetime.date(2021,6,15)))
# # plt.ylim((100,250))
# plt.show()

In [1039]:

# # ticker = yfinance.Ticker(name)
# # data = ticker.history(interval="1d",start=start_date,end=end_date)

# data['Forward Close'] = data['Close'].shift(-n_forward)

# data['Forward Return'] = (data['Forward Close'] - data['Close'])/data['Close']

In [ ]:
# result = []
# train_size = 0.6

# for sma_length in range(20,500):
  
#   data['SMA'] = data['Close'].rolling(sma_length).mean()
#   data['input'] = [int(x) for x in data['Close'] > data['SMA']]
  
#   df = data.dropna()

#   training = df.head(int(train_size * df.shape[0]))
#   test = df.tail(int((1 - train_size) * df.shape[0]))
  
#   tr_returns = training[training['input'] == 1]['Forward Return']
#   test_returns = test[test['input'] == 1]['Forward Return']

#   mean_forward_return_training = tr_returns.mean()
#   mean_forward_return_test = test_returns.mean()

#   pvalue = ttest_ind(tr_returns,test_returns,equal_var=False)[1]
 
#   result.append({
#       'sma_length':sma_length,
#       'training_forward_return': mean_forward_return_training,
#       'test_forward_return': mean_forward_return_test,
#       'p-value':pvalue
#   })

In [ ]:
# result.sort(key = lambda x : -x['training_forward_return'])

In [ ]:
# result[0]

In [ ]:
# best_sma = result[0]['sma_length']
# data['SMA'] = data['Close'].rolling(best_sma).mean()

In [ ]:
# plt.plot(data['Close'],label="fb")

# plt.plot(data['SMA'],label = "{} periods SMA".format(best_sma))

# plt.legend()
# plt.show()



In [ ]:

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=data['Time'], y=data['SMA'], name=f"{best_sma} periods SMA"))
# fig.add_trace(go.Scatter(x=data['Time'], y=data['Close']))

# fig.show()





In [ ]:
# candlestick = go.Candlestick(x=data['Time'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])

# fig = go.Figure(data=candlestick)

# fig.add_trace(go.Scatter(x=data['Time'], y=data['SMA'], name=f"{best_sma} periods SMA"))
# fig.layout.xaxis.rangeslider.visible = False

# fig.show()